### 1. data load

In [2]:
import pandas as pd

bj_aq = pd.read_csv('data/beijing_17_18_aq.csv')
bj_meo = pd.read_csv('data/beijing_17_18_meo.csv')
bj_his = pd.read_csv('data/Beijing_historical_meo_grid.csv')

### 2. make empty DataFrame

In [3]:
import datetime

bj = pd.DataFrame(columns=['station_id', 'utc_time'])

bj_aq = bj_aq.rename(columns={'stationId':'station_id'})  # stationId -> station_id로 바꿔주자
bj_station_list = list(set(bj_aq['station_id']))
bj_station_list = list(map(lambda x : x.replace('_aq', ''), bj_station_list))

start_time = datetime.datetime.strptime('2017-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
utc_time = [(start_time + datetime.timedelta(hours=i)).isoformat().replace('T', ' ') for i in range(600*24)]

bj['station_id'] = sorted(bj_station_list * 600 * 24)
bj['utc_time'] = utc_time * len(bj_station_list)

bj.head()

,station_id,utc_time
0,aotizhongxin,2017-01-01 00:00:00
1,aotizhongxin,2017-01-01 01:00:00
2,aotizhongxin,2017-01-01 02:00:00
3,aotizhongxin,2017-01-01 03:00:00
4,aotizhongxin,2017-01-01 04:00:00


### 3. merge air_quality

In [4]:
bj_aq['station_id'] = list(map(lambda x : x.replace('_aq', ''), bj_aq['station_id']))

bj_result = pd.merge(bj, bj_aq, on=['station_id', 'utc_time'])

bj_result.head()

,station_id,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
0,aotizhongxin,2017-01-01 14:00:00,453.0,467.0,156.0,7.2,3.0,9.0
1,aotizhongxin,2017-01-01 15:00:00,417.0,443.0,143.0,6.8,2.0,8.0
2,aotizhongxin,2017-01-01 16:00:00,395.0,467.0,141.0,6.9,3.0,8.0
3,aotizhongxin,2017-01-01 17:00:00,420.0,484.0,139.0,7.4,3.0,9.0
4,aotizhongxin,2017-01-01 18:00:00,453.0,520.0,157.0,7.6,4.0,9.0


In [5]:
bj_result.shape

(311010, 8)

### 4. merge historical_meo_grid

In [6]:
from utils import get_nearest_grid
bj_result['stationName'] = list(map(get_nearest_grid, bj_result['station_id']))

bj_result = pd.merge(bj_result, bj_his, on=['stationName', 'utc_time'])

bj_result.head()

,station_id,utc_time,PM2.5,PM10,NO2,CO,O3,SO2,stationName,longitude,latitude,temperature,pressure,humidity,wind_direction,wind_speed/kph
0,aotizhongxin,2017-01-01 14:00:00,453.0,467.0,156.0,7.2,3.0,9.0,beijing_grid_304,116.4,40.0,-2.47,1018.38,54.75,322.13,4.55
1,aotizhongxin,2017-01-01 15:00:00,417.0,443.0,143.0,6.8,2.0,8.0,beijing_grid_304,116.4,40.0,-2.27,1018.62,52.49,329.11,7.49
2,aotizhongxin,2017-01-01 16:00:00,395.0,467.0,141.0,6.9,3.0,8.0,beijing_grid_304,116.4,40.0,-2.01,1018.78,49.53,331.53,9.71
3,aotizhongxin,2017-01-01 17:00:00,420.0,484.0,139.0,7.4,3.0,9.0,beijing_grid_304,116.4,40.0,-1.75,1018.95,46.58,333.05,11.95
4,aotizhongxin,2017-01-01 18:00:00,453.0,520.0,157.0,7.6,4.0,9.0,beijing_grid_304,116.4,40.0,-1.49,1019.12,43.62,334.09,14.18


In [7]:
bj_result.shape

(311010, 16)

### 5. drop_na

In [8]:
bj_result = bj_result.dropna()

bj_result.shape

(200390, 16)

### 6. save table

In [10]:
bj_result.to_csv('beijing_table.csv')